In [1]:
import torch
from graspflow_classifiers import S_classifier, C_classifier
torch.manual_seed(4)

In [2]:
S = S_classifier(path_to_model='saved_models/evaluator/165228576343/100.pt', device='cpu')
C = C_classifier(device='cpu')

In [3]:
t=torch.rand([4,3], requires_grad=True)
r=torch.rand([4,3], requires_grad=True)
pc = torch.rand([4,1024,3])
pc_mean = torch.rand([4,3])

In [4]:
t=torch.nn.Parameter(t.clone())
r=torch.nn.Parameter(r.clone())
optimizer = torch.optim.SGD([{'params': [t]}], lr=1, maximize=True)
optimizer2 = torch.optim.SGD([{'params': [r]}], lr=1, maximize=True)

In [5]:
def step(losses, t, r):
    t_init = t.clone()
    r_init = r.clone()
    t_grads = []
    r_grads = []
    for loss in losses:
        optimizer.zero_grad()
        loss.backward(torch.ones_like(loss), retain_graph=True)
        #optimizer.step()
        #optimizer2.step()
        
         # extract delta change
        delta_t = optimizer.param_groups[0]['params'][0].detach().clone() - t_init
        delta_r = optimizer2.param_groups[0]['params'][0].detach().clone() - r_init
        
        delta_t = optimizer.param_groups[0]['params'][0].detach().clone() - t_init
        delta_r = optimizer2.param_groups[0]['params'][0].detach().clone() - r_init
        
        
        t_grads.append(delta_t)
        r_grads.append(delta_r)

        
        with torch.no_grad():
            optimizer.param_groups[0]['params'][0].copy_(t_init)
            optimizer2.param_groups[0]['params'][0].copy_(r_init)
        
    # make update
    t_new = t_init.clone()
    r_new = r_init.clone()

    for idx, classifier in enumerate('SC'):
        t_new = t_new + 0.1*t_grads[idx]
        r_new = r_new + 0.1*r_grads[idx]
        
    # assign to params
    with torch.no_grad():
        optimizer.param_groups[0]['params'][0].copy_(t_new)
        optimizer2.param_groups[0]['params'][0].copy_(r_new)
        
    return t_init

In [6]:
for k in range(3):
    
    print(pc.shape)
    print(pc_mean.shape)
    with torch.autograd.set_detect_anomaly(True):

        logit2, _ = S.forward(t=t, r=r, pc=pc, grasp_space='Euler')
        logit1, _ = C.forward(t=t, r=r, pc_mean=pc_mean, grasp_space='Euler')

        loss1 = torch.nn.functional.logsigmoid(logit1)
        loss2 = torch.nn.functional.logsigmoid(logit2)
        losses = [loss1, loss2]
        t_init = step(losses, t, r)

torch.Size([4, 1024, 3])
torch.Size([4, 3])


/home/tasbolat/some_python_examples/graspflow_models/graspflow_venv/lib/python3.8/site-packages/torch/autograd/__init__.py:173: UserWarning: Error detected in SinBackward0. Traceback of forward call that caused the error:
  File "/usr/lib/python3.8/runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.8/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/home/tasbolat/some_python_examples/graspflow_models/graspflow_venv/lib/python3.8/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/home/tasbolat/some_python_examples/graspflow_models/graspflow_venv/lib/python3.8/site-packages/traitlets/config/application.py", line 982, in launch_instance
    app.start()
  File "/home/tasbolat/some_python_examples/graspflow_models/graspflow_venv/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/home/tasbolat/some_python_ex

RuntimeError: Output 2 of UnbindBackward0 is a view and its base or another view of its base has been modified inplace. This view is the output of a function that returns multiple views. Such functions do not allow the output views to be modified inplace. You should replace the inplace operation by an out-of-place one.

In [16]:
def f1(x,y):
    return 2*x**2 + 3*y**3

In [17]:
def f2(x,y):
    return x+y

In [20]:
x = torch.FloatTensor([1.0])
y = torch.FloatTensor([1.0])

x = torch.nn.Parameter(x.clone())
y = torch.nn.Parameter(y.clone())

optimizer = torch.optim.SGD([{'params':[x]},{'params':[y]}], lr=0.1)

In [27]:
def step(losses, t, r):
    t_init = t.clone()
    r_init = r.clone()
    t_grads = []
    r_grads = []
    for loss in losses:
        print('start')
        optimizer.zero_grad()
        loss.backward(torch.ones_like(loss), retain_graph=True)
        
        print(optimizer.param_groups[0]['params'][0])
        optimizer.step()
        print(optimizer.param_groups[0]['params'][0])
         # extract delta change
        delta_t = optimizer.param_groups[0]['params'][0].detach().clone() - t_init
        delta_r = optimizer.param_groups[1]['params'][0].detach().clone() - r_init

        t_grads.append(delta_t)
        r_grads.append(delta_r)

        with torch.no_grad():
            optimizer.param_groups[0]['params'][0].copy_(t_init)
            optimizer.param_groups[1]['params'][0].copy_(r_init)
        
    # make update
    t_new = t_init.clone()
    r_new = r_init.clone()

    for idx, classifier in enumerate('SC'):
        t_new = t_new + 0.1*t_grads[idx]
        r_new = r_new + 0.1*r_grads[idx]
        
    # assign to params
    with torch.no_grad():
        optimizer.param_groups[0]['params'][0].copy_(t_new)
        optimizer.param_groups[1]['params'][0].copy_(r_new)
        
    print(optimizer.param_groups[0]['params'][0])
    print('end')    
    return t_init

In [28]:
for k in range(10):
    print(k)
    
    with torch.autograd.set_detect_anomaly(True):

        logit2 = f2(x,y)
        logit1 = f1(x,y)

        #loss1 = torch.nn.functional.logsigmoid(logit1)
        #loss2 = torch.nn.functional.logsigmoid(logit2)
        losses = [logit1, logit2]
        t_init = step(losses, x, y)

0
start
Parameter containing:
tensor([0.5679], requires_grad=True)
Parameter containing:
tensor([0.3408], requires_grad=True)
start
Parameter containing:
tensor([0.5679], requires_grad=True)
Parameter containing:
tensor([0.4679], requires_grad=True)
Parameter containing:
tensor([0.5352], requires_grad=True)
end
1
start
Parameter containing:
tensor([0.5352], requires_grad=True)
Parameter containing:
tensor([0.3211], requires_grad=True)
start
Parameter containing:
tensor([0.5352], requires_grad=True)
Parameter containing:
tensor([0.4352], requires_grad=True)
Parameter containing:
tensor([0.5038], requires_grad=True)
end
2
start
Parameter containing:
tensor([0.5038], requires_grad=True)
Parameter containing:
tensor([0.3023], requires_grad=True)
start
Parameter containing:
tensor([0.5038], requires_grad=True)
Parameter containing:
tensor([0.4038], requires_grad=True)
Parameter containing:
tensor([0.4736], requires_grad=True)
end
3
start
Parameter containing:
tensor([0.4736], requires_grad=

In [33]:
import itertools
all_classifiers = [''.join(l) for i in range(len(classifiers)) for l in itertools.combinations(classifiers, i+1)]

In [42]:
all_classifiers

['S',
 'C',
 'E',
 'T',
 'N',
 'SC',
 'SE',
 'ST',
 'SN',
 'CS',
 'CE',
 'CT',
 'CN',
 'ES',
 'EC',
 'ET',
 'EN',
 'TS',
 'TC',
 'TE',
 'TN',
 'NS',
 'NC',
 'NE',
 'NT',
 'SCE',
 'SCT',
 'SCN',
 'SEC',
 'SET',
 'SEN',
 'STC',
 'STE',
 'STN',
 'SNC',
 'SNE',
 'SNT',
 'CSE',
 'CST',
 'CSN',
 'CES',
 'CET',
 'CEN',
 'CTS',
 'CTE',
 'CTN',
 'CNS',
 'CNE',
 'CNT',
 'ESC',
 'EST',
 'ESN',
 'ECS',
 'ECT',
 'ECN',
 'ETS',
 'ETC',
 'ETN',
 'ENS',
 'ENC',
 'ENT',
 'TSC',
 'TSE',
 'TSN',
 'TCS',
 'TCE',
 'TCN',
 'TES',
 'TEC',
 'TEN',
 'TNS',
 'TNC',
 'TNE',
 'NSC',
 'NSE',
 'NST',
 'NCS',
 'NCE',
 'NCT',
 'NES',
 'NEC',
 'NET',
 'NTS',
 'NTC',
 'NTE',
 'SCET',
 'SCEN',
 'SCTE',
 'SCTN',
 'SCNE',
 'SCNT',
 'SECT',
 'SECN',
 'SETC',
 'SETN',
 'SENC',
 'SENT',
 'STCE',
 'STCN',
 'STEC',
 'STEN',
 'STNC',
 'STNE',
 'SNCE',
 'SNCT',
 'SNEC',
 'SNET',
 'SNTC',
 'SNTE',
 'CSET',
 'CSEN',
 'CSTE',
 'CSTN',
 'CSNE',
 'CSNT',
 'CEST',
 'CESN',
 'CETS',
 'CETN',
 'CENS',
 'CENT',
 'CTSE',
 'CTSN',
 'CTES',